In [334]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [335]:
import warnings
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from oracle import make_oracle
from optimize import optimize_newton, optimize_gd, optimize_hfn

from sklearn.linear_model import LogisticRegression

warnings.filterwarnings('ignore')
np.random.seed(0)

In [352]:
def plot_model(oracle, optimizers_param):
    model = LogisticRegression(penalty="none", tol=1e-8, max_iter=10000, n_jobs=-1, fit_intercept=False)
    model.fit(oracle.X, oracle.y.ravel())
    
    entropy_true = oracle.value(model.coef_.reshape(-1, 1))
    
    w_n = oracle.X.shape[1]
    w_init = np.random.uniform(size=(w_n, 1))
    
    fig, axs = plt.subplots(3, 1, figsize=(12, 18))
    
    for i, (name, log_feature, optimizer, line_search_methods, log_modulo) in enumerate(optimizers_param):
        for line_search in line_search_methods:        
            w, log = optimizer(oracle, w_init, line_search_method=line_search, log_modulo=log_modulo)
            
            if log_feature == "n_iters":
                feature = np.arange(log.get_log()["time"].shape[0]) * log_modulo
            else:
                feature = log.get_log()[log_feature]
            
            entropy = log.get_log()["entropy"]

            axs[i].plot(feature, np.log(np.abs(entropy - entropy_true)), 
                        label=f"{line_search}: {np.round(np.min(entropy), 4)}")
    
    for ax, param in zip(axs.flat, optimizers_param):
        print(log_feature)
        ax.set_title(param[0])
        ax.set(xlabel=f"{log_feature}", ylabel="absolute error")
        ax.legend()

# a1a dataset

In [358]:
a1a_oracle = make_oracle()

a1a_optim_params = [
    ("gradien descent", "n_iters", optimize_gd, ["golden", "brent", "armijo", "wolfe", "lipschitz"], 100),
#     ("newton", "n_iters", optimize_newton, ["golden", "brent", "armijo", "wolfe"], 1),
#     ("hessian-free newton", "n_iters", optimize_hfn, ["golden", "brent", "armijo", "wolfe"], 1)
]

In [ ]:
plot_model(a1a_oracle, a1a_optim_params)